# Как переводить

- Xcode / Editor / Export for Localization / (Repository-root)/CDDDA-loc.
- Translations.csv / Copy "en" column / Google translate from english to target language.
- Translations.csv / New column with header named after lang, like "ru" without quotes.
- Translations.csv / Paste appropriate translations to appropriate lines.
- l10n.ipynb (this notebook) / Run all cells, fix errors in CSV sanity check.
- l10n.ipynb (this notebook) / Check save log, that your language is present.
- Xcode / Editor / Import Localizations / Select your localization.
- Simulator / Change language of device to target one.
- Xcode / Run in simulator. Check that screen menu is appropriately localized.

In [1]:
import csv
import gettext
import pathlib
import itertools
import os

import lxml.etree as etree
import pandas as pd

In [2]:
l10n_root = pathlib.Path('../CDDA-loc')
l10n_csv = pathlib.Path('../Translations.csv')
wrong_directory = l10n_root / 'CDDA-loc'
locale_dir = pathlib.Path('../Libraries/Cataclysm-DDA/lang/mo')
languages = 'en	hu	es	zh	ko	de	pt_BR	ru	fr	ja'.split('\t')

gettext.bindtextdomain('cataclysm-dda', localedir=locale_dir)
gettext.textdomain('cataclysm-dda')

problematic_lines_to_strings_for_translation_and_handlers = {
    '* Toggle Snap-to-target': ('[%c] target self; [%c] toggle snap-to-target', lambda t: t.rsplit('] ', 1)[1]),
    'm Change Aim Mode': ('[%c] to switch aiming modes.', lambda t: t.replace('[%c] ', '')),
    's Toggle Burst/Auto Mode': ('[%c] to switch firing modes.', lambda t: t.replace('[%c] ', '')),
}

xliffs = list(l10n_root.glob('*/Localized Contents/*.xliff'))
xliffs

[PosixPath('../CDDA-loc/pt.xcloc/Localized Contents/pt.xliff'),
 PosixPath('../CDDA-loc/fr.xcloc/Localized Contents/fr.xliff'),
 PosixPath('../CDDA-loc/zh.xcloc/Localized Contents/zh.xliff'),
 PosixPath('../CDDA-loc/hu.xcloc/Localized Contents/hu.xliff'),
 PosixPath('../CDDA-loc/ru.xcloc/Localized Contents/ru.xliff'),
 PosixPath('../CDDA-loc/ko.xcloc/Localized Contents/ko.xliff'),
 PosixPath('../CDDA-loc/en.xcloc/Localized Contents/en.xliff'),
 PosixPath('../CDDA-loc/de.xcloc/Localized Contents/de.xliff'),
 PosixPath('../CDDA-loc/ja.xcloc/Localized Contents/ja.xliff'),
 PosixPath('../CDDA-loc/es.xcloc/Localized Contents/es.xliff')]

In [17]:
def csv_sanity_check():
    with open(l10n_csv) as f:
        reader = csv.reader(f)
        langs = next(reader)
        errors = 0
        line = 1

        for record in reader:
            line += 1
            source = record[0]
            if (len(source) == 1) or (source[1] == ' '):
                sym = source[0]

                for lang, txn in zip(langs, record):
                    if not txn:
                        continue
                    if txn[0] != sym:
                        errors += 1
                        print(f'{line}: Symbol {sym} differs for language {lang}: {txn}')
                    if (len(source) == 1) and (len(txn) != 1):
                        errors += 1
                        print(f'{line}: Trailing symbols for {sym} in {txn}')
    return errors


def findall(tree, tag):
    return tree.findall(f'.//{{urn:oasis:names:tc:xliff:document:1.2}}{tag}')


def save_translation(xliff: pathlib.Path):
    with open(xliff) as f:
        tree = etree.parse(f)
    
    with open(l10n_csv) as f:
        translations_csv_reader = csv.reader(f)
        langs = next(translations_csv_reader)[1:]
        translation_mappings = {row[0].lower(): dict(zip(langs, row[1:])) for row in translations_csv_reader}
    
    lang = xliff.name.split('.')[0]
    trans_units = findall(tree, 'trans-unit')
    errors = 0
    
    for trans_unit in trans_units:
        source = findall(trans_unit, 'source')[0]
        
        try:
            translation = translation_mappings[source.text.lower()][lang]
        except KeyError:
            print(f'txn for {lang}/{source.text} not found. Skipping.')
            errors += 1
            continue

        try:
            target = findall(trans_unit, 'target')[0]
        except IndexError:
            target = etree.SubElement(trans_unit, 'target')

        target.text = translation
        
    tree.write(str(xliff), encoding='utf8')

    return errors

    
def get_targets_from_xliff(xliff):
    with open(xliff) as f:
        tree = etree.parse(f)
    
    targets = (x.text for x in findall(tree, 'target'))
    return targets


def translate(text: str):
    for language_code in languages:
        os.environ['LANGUAGE'] = language_code
        yield language_code.split('_')[0], gettext.gettext(text)


def get_texts_from_csv():
    with open(l10n_csv) as f:
        reader = csv.reader(f)
        langs = next(reader)
        for record in reader:
            yield record[0]


def get_texts_for_translation():
    english_texts = get_texts_from_csv()
    
    for text in english_texts:
        if len(text) == 1:
            continue
        elif text in problematic_lines_to_strings_for_translation_and_handlers:
            prefix = text[:2]
            real_text, processing_function = problematic_lines_to_strings_for_translation_and_handlers[text]
        elif text[1] == ' ':
            prefix, real_text = text[:2], text[2:]
            processing_function = None
        else:
            prefix = ''
            real_text = text
            processing_function = None

        for language, translation in translate(real_text):
            yield (text, language, prefix + (processing_function(translation) if processing_function else translation))


def analyze(df):
    for text, subframe in df.groupby(df.text):
        if subframe.describe().translation['unique'] == 1:
            status = 0
        else:
            status = 1

        yield text, status


def save_translations():
    errors = 0
    for xliff in xliffs:
        if str(xliff).endswith('en.xliff'):
            continue
        print(f'Saving {xliff}')
        errors += save_translation(xliff)
    return errors


assert not csv_sanity_check(), 'CSV file has errors'
assert not wrong_directory.is_dir(), f'Another localization directory {wrong_directory} found inside of the correct. To prevent mistakes, please delete both and then start from exporting from Xcode.'

In [4]:
print('\n'.join(get_targets_from_xliff([x for x in xliffs if str(x).endswith('en.xliff')][0])))

N Toggle minimap
1
U Unload or empty wielded item
m View map
Info
7
L Move view East
0 View help
: Center view
Look
4
D
R Read
Y Manage zones
Inventory
V List all items around the player
I
W
A Apply or use wielded item
/ Advanced inventory management
v View morale
{ Toggle map memory
Misc
6
3
9
SPACE
a Apply or use item
& Craft items
_ Select martial arts style
} Toggle panel admin
+ Re-layer armor/clothing
Combat
2
SPACE
P View message log
X Peek around corners
[ View/activate mutations
= Swap inventory letters
G Grab something nearby
K Move view North
( Disassemble items
e Examine nearby terrain
) View scores
c Close door
| Wait for several minutes
> Descend stairs
D Drop item to adjacent tile
- Recraft last recipe
TAB
" Movement mode menu
J Move view South
ESC
i Open inventory
M View missions
s Toggle burst/auto mode
m Change aim mode
* Toggle snap-to-target
f Fire wielded item
5
w Wield
r Reload item
< Ascend stairs
⮐
C
d Drop item
8
H Move view West
Craft
x Look around
@ View play

In [5]:
print('\n'.join('\t'.join(x) for x in translate('Re-layer armor/clothing')))

en	Re-layer armor/clothing
hu	Páncélzat és ruha újrarétegezése
es	Reordenar armadura/ropa
zh	整理装束
ko	보호구/의류 재정렬하기
de	Bekleidungsschichten umsortieren
pt	Alterar Camadas de armadura/roupa
ru	Очерёдность брони/одежды
fr	Organiser les armures
ja	防具/衣類の階層を変更


In [6]:
df = pd.DataFrame.from_records(get_texts_for_translation(), columns='text language translation'.split())
df

,text,language,translation
0,N Toggle Minimap,en,N Toggle Minimap
1,N Toggle Minimap,hu,N Toggle Minimap
2,N Toggle Minimap,es,N Alternar minimapa
3,N Toggle Minimap,zh,N 切换小地图
4,N Toggle Minimap,ko,N Toggle Minimap
...,...,...,...
855,CDDA,de,CDDA
856,CDDA,pt,CDDA
857,CDDA,ru,CDDA
858,CDDA,fr,CDDA


In [7]:
df.describe()

,text,language,translation
count,860,860,860
unique,84,10,718
top,SPACE,pt,SPACE
freq,30,86,30


In [8]:
status_df = pd.DataFrame.from_records(analyze(df), columns='text status'.split())
status_df[status_df.status == 0]

,text,status
22,BTAB,0
24,CDDA,0
25,Cataclysm RPG,0
30,ESC,0
46,Overlay UI enabled,0
51,SPACE,0
53,TAB,0


In [9]:
untranslated_ok_strings = """Overlay UI enabled
Cataclysm RPG
CDDA
BTAB
TAB
ESC
SPACE""".split('\n')


assert len(status_df[status_df.status == 0]) == len(untranslated_ok_strings), f'Too many strings left untranslated: {len(status_df[status_df.status == 0])}, expected {len(untranslated_ok_strings)}'

In [10]:
csv_df = pd.read_csv(l10n_csv).set_index('en')
csv_df

,hu,es,zh,ko,de,pt,ru,fr,ja
en,,,,,,,,,
N Toggle Minimap,N Váltás a minimapon,N Minimapa de alternancia,N切换小地图,N 미니 맵 전환,N Minikarte umschalten,N Alternar minimapa,N Переключить миникарту,N Basculer la mini-carte,Nミニマップを切り替えます
1,1,1,1,1,1,1,1,1,1
U Unload or Empty Wielded Item,U Rakja ki vagy ürítse ki az árnyékolt tárgyat,U Descargar o vaciar el artículo empuñado,U卸货或空置的物品,U 사용 된 아이템을 내리거나 비 웁니다.,U Entladen oder leeren Gegenstand,U Descarregue ou esvazie o item empunhado,U Разряженный или пустой предмет в руках,U Décharger ou vider l'objet utilisé,U使用アイテムをアンロードまたは空にする
m View Map,m Térkép megtekintése,m Ver mapa,m查看地图,m지도보기,m Karte anzeigen,m Ver mapa,m Посмотреть карту,m Voir la carte,m地図を見る
Info,Info,Información,信息,정보,Die Info,Informações,Информация,Info,情報
...,...,...,...,...,...,...,...,...,...
BTAB,BTAB,BTAB,BTAB,BTAB,BTAB,BTAB,BTAB,BTAB,BTAB
O Zone activities,O Zóna tevékenységek,O Actividades de la Zona,O区活动,O Zone 활동,O Zonenaktivitäten,O Atividades da Zona,O зоны деятельности,O Activités de la zone,Oゾーンの活動
Overlay UI enabled,Az overlay felhasználói felület engedélyezve,IU de superposición habilitada,启用重叠式用户界面,오버레이 UI 사용,Overlay-Benutzeroberfläche aktiviert,Overlay UI habilitado,Пользовательский интерфейс наложения включен,Interface utilisateur de superposition activée,オーバーレイUIが有効


In [11]:
for key, record in df.iterrows():
    text, language, translation = record
    if translation.lower() != text.lower():
        csv_df.loc[text][language] = translation
csv_df

,hu,es,zh,ko,de,pt,ru,fr,ja
en,,,,,,,,,
N Toggle Minimap,N Váltás a minimapon,N Alternar minimapa,N 切换小地图,N 미니 맵 전환,N Minikarte umschalten,N Alternar minimapa,N Миникарта,N Basculer la mini-carte,N 切替/ミニマップ表示
1,1,1,1,1,1,1,1,1,1
U Unload or Empty Wielded Item,U Kézben fogott tárgy kiürítése,U Descargar/Vaciar obj. empuñado,U 卸载/清空手持物品,U 손에 든 물품의 내용물/충전물 비우기,U Gehaltenen Gegenstand entladen/leeren,U Descarregar ou Esvaziar Item Empunhado,U Разрядить/опустошить предмет в руках,U Décharger ou vider l'objet en main,U 装填物の抜き取り(装備中)
m View Map,m Térkép megtekintése,m Ver Mapa,m 查看地图,m 지도 보기,m Karte anzeigen,m Ver Mapa,m Карта,m Voir la carte,m マップ
Info,Infó,Información,说明信息,정보,Die Info,Informações,Информация,Info,情報
...,...,...,...,...,...,...,...,...,...
BTAB,BTAB,BTAB,BTAB,BTAB,BTAB,BTAB,BTAB,BTAB,BTAB
O Zone activities,O Zóna aktivitás,O Zona de actividades,O 区域活动,O Zone 활동,O Zonenaktivitäten ausführen,O Atividades de zona,O Действия с зонами,O Activités de la zone,O 区域作業
Overlay UI enabled,Az overlay felhasználói felület engedélyezve,IU de superposición habilitada,启用重叠式用户界面,오버레이 UI 사용,Overlay-Benutzeroberfläche aktiviert,Overlay UI habilitado,Пользовательский интерфейс наложения включен,Interface utilisateur de superposition activée,オーバーレイUIが有効


In [12]:
csv_df.to_csv(l10n_csv)

In [19]:
assert not save_translations(), 'There are errors in translations, better fix them to get product of high quality.'

Saving ../CDDA-loc/pt.xcloc/Localized Contents/pt.xliff
txn for pt/g Pickup nearby item(s) not found. Skipping.
Saving ../CDDA-loc/fr.xcloc/Localized Contents/fr.xliff
txn for fr/g Pickup nearby item(s) not found. Skipping.
Saving ../CDDA-loc/zh.xcloc/Localized Contents/zh.xliff
txn for zh/g Pickup nearby item(s) not found. Skipping.
Saving ../CDDA-loc/hu.xcloc/Localized Contents/hu.xliff
txn for hu/g Pickup nearby item(s) not found. Skipping.
Saving ../CDDA-loc/ru.xcloc/Localized Contents/ru.xliff
txn for ru/g Pickup nearby item(s) not found. Skipping.
Saving ../CDDA-loc/ko.xcloc/Localized Contents/ko.xliff
txn for ko/g Pickup nearby item(s) not found. Skipping.
Saving ../CDDA-loc/de.xcloc/Localized Contents/de.xliff
txn for de/g Pickup nearby item(s) not found. Skipping.
Saving ../CDDA-loc/ja.xcloc/Localized Contents/ja.xliff
txn for ja/g Pickup nearby item(s) not found. Skipping.
Saving ../CDDA-loc/es.xcloc/Localized Contents/es.xliff
txn for es/g Pickup nearby item(s) not found. Sk

AssertionError: There are errors in translations, better fix them to get product of high quality.